In [7]:
from pathlib import Path
import os
this_directory = Path(os.getcwd()).resolve()
project_directory = this_directory.parent
import sys
sys.path.append(project_directory.as_posix())
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession\
        .builder\
        .appName("test")\
        .getOrCreate()
from random import random
from operator import add
from pyspark.sql.types import *
schema = StructType([
 StructField("index", IntegerType(), False),
 StructField("in_time", TimestampType(), False),
 StructField("out_time", TimestampType(), False),
 StructField("berthage", IntegerType(), False),
 StructField("section", StringType(), False),
 StructField("admin_region", StringType(), False), 
 StructField("parking_time", LongType(), False),
 ])
data_path = (project_directory/"data/clean_parking_data.csv").as_posix()
df = spark.read.csv(data_path,
                    header=True, 
                    inferSchema=True,
                    
                    )
                #     schema=schema)
df.limit(5).show(5)

+-----+-------------------+-------------------+--------+----------------+------------+------------+
|index|            in_time|           out_time|berthage|         section|admin_region|parking_time|
+-----+-------------------+-------------------+--------+----------------+------------+------------+
|    0|2018-09-01 10:10:00|2018-09-01 12:00:00|  201091|荔园路(蛇口西段)|      南山区|        6600|
|    1|2018-09-01 13:43:35|2018-09-01 14:29:35|  201091|荔园路(蛇口西段)|      南山区|        2760|
|    2|2018-09-01 15:10:54|2018-09-01 16:08:54|  201091|荔园路(蛇口西段)|      南山区|        3480|
|    3|2018-09-01 16:34:03|2018-09-01 17:56:03|  201091|荔园路(蛇口西段)|      南山区|        4920|
|    4|2018-09-01 18:40:20|2018-09-01 20:00:20|  201091|荔园路(蛇口西段)|      南山区|        4800|
+-----+-------------------+-------------------+--------+----------------+------------+------------+



In [28]:
sc = spark.sparkContext
rdd = df.rdd
# 统计总的停车时间
total_parking_time = rdd.map(lambda x: int(x[-1])).reduce(lambda x, y: x + y)
# 计算平均停车时间
avg_parking_time = total_parking_time / rdd.count()
# 统计每个区域的停车时间
parking_time_by_region = rdd.map(lambda x: (x[-2], int(x[-1]))) \
    .reduceByKey(lambda x, y: x + y)
total_parking_time

3363310920

In [26]:
# 构建答案空间
sf = sc.range(0, 48, 1).map(lambda x: (x, x+1))
sf.take(5)
sections = df.select('section').distinct().rdd
sections.take(5)
sfs = sf.cartesian(sections)
sfs.take(5)
sfs.map(lambda x: (x[0][0], x[0][1], x[1][0], 0)).take(5)

[(0, 1, '招商路(蛇口西段)', 0),
 (0, 1, '荔园路(蛇口西段)', 0),
 (0, 1, '四海路', 0),
 (0, 1, '金世纪路', 0),
 (0, 1, '海月路(东段)', 0)]

In [ ]:
def process(x):
    
    return x
rdd.map(process).take(5)

In [27]:
sc.range(0, 5, 1).flatMap(lambda x:[x, x+1]).take(10)

[0, 1, 1, 2, 2, 3, 3, 4, 4, 5]

In [29]:
df.columns

['index',
 'in_time',
 'out_time',
 'berthage',
 'section',
 'admin_region',
 'parking_time']

In [32]:
rdd.take(1)

[Row(index=0, in_time=datetime.datetime(2018, 9, 1, 10, 10), out_time=datetime.datetime(2018, 9, 1, 12, 0), berthage=201091, section='荔园路(蛇口西段)', admin_region='南山区', parking_time=6600)]

In [46]:
import datetime
q = datetime.datetime(2018, 9, 1, 10, 10, 10 ) # 10点10分10秒
base = q.replace(hour=0, minute=0, second=0, microsecond=0)
d = q-base
# dir(d)
d.total_seconds()//1800

20.0

In [ ]:
def process(record):
    in_time = record[1]
    out_time = record[2]
    berthage = record[3]
    section = record[4]
    admin_region = record[5]
    # parking_time_half_hour = record[6]/1800 # 有几个半小时
    # for i in range(parking_time_half_hour):
    #     in_time
    base_date = in_time.replace(hour=0, minute=0, second=0, microsecond=0)
    in_half_hour = (in_time-base_date).total_seconds()//1800
    out_half_hour = (out_time-base_date).total_seconds()//1800
    
    for h in range(in_half_hour, out_half_hour+1):
        
    # 返回一个列表，表示这条记录作为一个berthage，对与哪些半小时

s = rdd.flatMap(process)
# 经过操作，s应该是(key, value)的形式，key是(start_half_hour, end_half_hour, section)
# value是(berthage, contributions)

In [ ]:
s = s.distinct(lambda v:v[1]) \
    .map(lambda x: (x[0], x[1][1])) # 对berthage做去重，贡献

In [63]:
# key 是 (start_half_hour, end_half_hour, section)
# s 是一堆 (key, 1) 或者 (key, 2) 的集合
res = s.reduceByKey(lambda x, y: x+y)
# res现在是 每一个 (start_half_hour, end_half_hour, section) 对应的总共的出现过的独立停车场数。
# 从后往前构造

NameError: name 's' is not defined

In [ ]:
section_berthage_rdd = rdd.map(lambda x: ((x[4], x[3]), 1))


In [60]:
# 假设rdd为停车记录数据集，变量名为parking_data
# 生成(section, berthage)键值对的rdd
section_berthage_rdd = rdd.map(lambda x: ((x[4]),  x[3]))
# 按section分组
grouped_by_section_rdd = section_berthage_rdd.groupByKey()
# grouped_by_section_rdd.take(5)
grouped_by_section_rdd.mapValues(lambda x: len(set(x)))\
    .sortBy(lambda x: x[1], ascending=False)\
    .take(5)

[('高新南环路', 240),
 ('科技南一路', 92),
 ('科技南路', 87),
 ('后海大道辅道(后海段)', 80),
 ('高新南十道', 73)]

In [65]:
section_berthage_rdd = rdd.map(lambda x: ((x[4]),  x[3]))
# count_rdd = section_berthage_rdd.distinct(lambda x:x) \
count_rdd = section_berthage_rdd.distinct() \
    .map(lambda x: (x[0], 1)) \
    .reduceByKey(lambda a, b: a + b)
top_counts = count_rdd.sortBy(lambda x: x[1], ascending=False).take(5)
top_counts

[('高新南环路', 240),
 ('科技南一路', 92),
 ('科技南路', 87),
 ('后海大道辅道(后海段)', 80),
 ('高新南十道', 73)]

In [59]:
result_df = df.groupBy("section")\
    .agg(countDistinct("berthage").alias("berthage_count"))\
    .sort(col("berthage_count").desc())
result_df.show(5)

+--------------------+--------------+
|             section|berthage_count|
+--------------------+--------------+
|          高新南环路|           240|
|          科技南一路|            92|
|            科技南路|            87|
|后海大道辅道(后海段)|            80|
|          高新南十道|            73|
+--------------------+--------------+
only showing top 5 rows

